# OCR用にPDFを画像化

In [1]:
from pathlib import Path
import pdf2image

OUTPUT_PATH = Path("output")
# open pdf and convert to pngs into a folder
for pdf_file in Path('.').glob('*.pdf'):
    images = pdf2image.convert_from_path(pdf_file)
    for i, image in enumerate(images):
        path = Path(OUTPUT_PATH / pdf_file.stem / f"{i}.png")
        path.parent.mkdir(parents=True, exist_ok=True)
        image.save(path)
    print(f"Converted {pdf_file} to pngs")

Converted sem1608.pdf to pngs


## 画像を裁断


In [4]:
import json
from PIL import Image

CROP_SRC_PATH = Path("crop")

for crop_path in CROP_SRC_PATH.glob("*.json"):
    with open(crop_path, "r") as file:
        crop_data = json.load(file)

    image_dir = Path("output") / crop_path.stem

    cropped_image_dir = Path("output") / crop_path.stem / "cropped"
    for file in cropped_image_dir.glob("**/*"):
        if file.is_file():
            file.unlink()
    cropped_image_dir.mkdir(parents=True, exist_ok=True)

    pages = sorted(image_dir.glob("*.png"))

    for page in pages:
        image = Image.open(page)
        image_width, image_height = image.size
        page_id = int(page.stem)
        for i, crop in enumerate(crop_data["crops"]):
            page_from = int(crop.get("page_from", 0))
            page_to = int(crop.get("page_to", len(pages)))

            if page_from > page_id:
                break

            if page_to < page_id:
                continue

            crop_dir = cropped_image_dir / f"{i}"
            crop_dir.mkdir(parents=True, exist_ok=True)

            cropped_image = image.crop((crop["x"] * image_width, crop["y"] * image_height, (crop["x"] + crop["width"]) * image_width, (crop["y"] + crop["height"]) * image_height))
            cropped_image.save(crop_dir / f"{page_id}.png")
